In [5]:
from skimage.color import rgb2gray, rgb2hsv
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
import math
import os
import sys

In [6]:
res_string =''
# Path to the folder with characters
directory = 'D:/try'
res_array = np.zeros((27,1))
string_array = np.zeros(8,dtype=int)
hyphen1 = 0
hyphen2 = 0
char_len = np.zeros(8)
#list of all the possible characters
plate_char = ["0","1","2","3","4","5","6","7","8","9","B","D","F","G","H","J","K","L","M","N","P","R","S","T","V","X","Z","-"]

##### Here we find the hyphens
##### We look for two smallest characters
for ind, filename in enumerate(os.listdir(directory)):
    detect = cv2.imread(directory +'/' + filename)
    char_len[ind] = len(detect)
    
hyphen1 = np.argmin(char_len) 
char_len[hyphen1] = 200
hyphen2 = np.argmin(char_len)  
#####

for ind, filename in enumerate(os.listdir(directory)):
    index  = 0
    detect = cv2.imread(directory +'/' + filename)

    ##Loop through all images and see which allign the most
    for i in range(len(plate_char)-1):
        
        # if the image is one of the hyphens - we put index to 27
        if(ind == hyphen1 or ind == hyphen2):
            index = 27
            break
        else:            
            orig = cv2.imread('dataset/'+ str(i) + '.jpg')  # character from the dataset
            dim = (len(orig[0]), len(orig)) 
            resized = cv2.resize(detect, dim)               # change the size of the character we want to recognize          
            bitwise = cv2.bitwise_and(resized,orig)         # perform biwise operation
            res = np.mean(bitwise[:,:])                     # compute mean of the resulting picture
            res_array[i] = np.abs(res - np.mean(resized[:,:])); 
        
        
    #choose the most appropriate character
    if(index == 0): # if the character is not a hyphen
        
        if(ind == 0):
            index = np.argmin(res_array) # take the smallest value of the res_array (means the best match)
        else:
            if(string_array[ind-1] != 27): # previous char is not a hyphen
                if(string_array[ind-1] in range(0,10)):
                    # the previous char is a number
                    index = np.argmin(res_array[:10])
                    
                else:
                    # the previous char is a letter
                    index = np.argmin(res_array[10:27]) + 10
                    
            else:  # previous char is a hyphen
                if(ind == 7 and string_array[1] in range(10,27) and string_array[5] in range(0,10)):  # VD-020-P(make sure the last character is a letter)
                    index = np.argmin(res_array[10:27]) + 10
                if(ind == 7 and string_array[1] in range(0,10) and string_array[5] in range(10,27)):  # 92-LHR-6 (make sure the last character is a number)                
                    index = np.argmin(res_array[:10])
                elif(ind == 6 and string_array[1] in range(0,10) and string_array[4] in range(10,27)): # plates like 89-LZ-GX (make sure the last two characters are letters)
                    index = np.argmin(res_array[10:27]) + 10
                elif(ind == 6 and string_array[1] in range(10,27) and string_array[4] in range(10,27)): # plates like LZ-GX-89 (make sure the last two characters are numbers)
                    index = np.argmin(res_array[:10])                    
                elif(string_array[ind-2] in range(0,10)): # we had numbers before hyphen 
                    index = np.argmin(res_array[10:27]) + 10
                else:
                    index = np.argmin(res_array) # take the smallest value of the res_array
    string_array[ind] = index
    
    
for i in string_array:
    res_string = res_string + plate_char[i]
print(res_string)

41-JS-SB
